In [4]:
# Import packages

import numpy
import os


In [5]:
# Change working directory

current_dir = os.getcwd()
current_dir

'/workspace/Mildew-Detection-in-Cherry-Leaves-P5/jupyter_notebooks'

In [6]:
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")

You set a new current directory


In [7]:
current_dir = os.getcwd()
current_dir

'/workspace/Mildew-Detection-in-Cherry-Leaves-P5'

In [8]:
# install kaggle
!pip install kaggle

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.0/59.0 kB 6.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.4/78.4 kB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.2/78.2 kB 31.1 MB/s eta 0:00:00
  Created wheel for kaggle: filename=kaggle-1.5.12-py3-none-any.whl size=73031 sha256=1727da7e163dd0f379fd1dfc19d8f57029f05e54f5c612b60e19b0377df7b152
  Stored in directory: /home/gitpod/.cache/pip/wheels/29/da/11/144cc25aebdaeb4931b231e25fd34b394e6a5725cbb2f50106
Successfully built kaggle


In [18]:
# Setting the kaggle configuration directory to the current working directory

os.environ["KAGGLE_CONFIG_DIR"] = os.getcwd()
! chmod 600 kaggle.json

In [19]:
# kaggle dataset and download

KaggleDatasetPath = "datasets/codeinstitute/cherry-leaves"
DestinationFolder = "inputs/cherry_leaves-dataset"
! kaggle datasets download -d {KaggleDatasetPath} -p {DestinationFolder}

401 - Unauthorized
